In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers
!pip install torch
!pip install deeppavlov
import numpy as np
from sklearn.metrics import precision_recall_fscore_support

     |████████████████████████████████| 3.8 MB 5.3 MB/s 
     |████████████████████████████████| 895 kB 23.2 MB/s 
     |████████████████████████████████| 67 kB 4.4 MB/s 
     |████████████████████████████████| 596 kB 41.8 MB/s 
     |████████████████████████████████| 6.5 MB 35.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 880 kB 5.1 MB/s 
     |████████████████████████████████| 57 kB 4.1 MB/s 
     |████████████████████████████████| 2.1 MB 44.7 MB/s 
     |████████████████████████████████| 8.2 MB 24.8 MB/s 
     |████████████████████████████████| 20.1 MB 1.4 MB/s 
     |████████████████████████████████| 10.4 MB 19.3 MB/s 
     |████████████████████████████████| 7.3 MB 3.1 MB/s 
     |████████████████████████████████| 1.5 MB 32.7 MB/s 
     |████████████████████████████████| 6.7 MB 24.1 MB/s 
     |████████████████████████████████| 859 k

In [ ]:
!cp  drive/MyDrive/НИР/dataset_bert.py .
!cp  drive/MyDrive/НИР/functions_for_bert.py .

In [ ]:
from dataset_bert import CustomDataset
from functions_for_bert import Classifier

In [ ]:
from transformers import DistilBertTokenizer
from transformers import DistilBertForSequenceClassification

device = 'cuda'

tokenizer_path = 'DeepPavlov/distilrubert-tiny-cased-conversational'
tokenizer = DistilBertTokenizer.from_pretrained(tokenizer_path)

model_path = 'DeepPavlov/distilrubert-tiny-cased-conversational'
model = DistilBertForSequenceClassification.from_pretrained(model_path)
model_ef = DistilBertForSequenceClassification.from_pretrained(model_path)

/usr/local/lib/python3.7/dist-packages/cryptography/hazmat/backends/openssl/x509.py:17: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  utils.DeprecatedIn35,


Downloading:   0%|          | 0.00/1.34M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/538 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/409M [00:00<?, ?B/s]

Some weights of the model checkpoint at DeepPavlov/distilrubert-tiny-cased-conversational were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_projector.weight', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at DeepPavlov/distilrubert-tiny-cased-conversational and are newly initialized: ['cl

In [ ]:
import torch

out_features = 768
classifier_DB = Classifier(
    model,
    tokenizer,
    out_features,
    6
)

classifier_DB_ef = Classifier(
    model_ef,
    tokenizer,
    out_features,
    6
)

In [ ]:
from deeppavlov.dataset_readers.basic_classification_reader import BasicClassificationDatasetReader
from deeppavlov.dataset_iterators.basic_classification_iterator import BasicClassificationDatasetIterator

# read data from particular columns of `.csv` file
dr = BasicClassificationDatasetReader().read(
    data_path='/content/drive/MyDrive/НИР/',
    train='Join_dataset.csv',
    x = 'text',
    y = 'news'
)

# initialize data iterator splitting `train` field to `train` and `valid` in proportion 0.8/0.2
train_iterator = BasicClassificationDatasetIterator(
    data=dr,
    field_to_split='train',  # field that will be splitted
    split_fields=['train', 'valid', 'test'],   # fields to which the fiald above will be splitted
    split_proportions=[0.7, 0.2, 0.1],  #proportions for splitting
    split_seed=23,  # seed for splitting dataset
    seed=42)  # seed for iteration over dataset

2022-03-17 17:43:54.276 WARNING in 'deeppavlov.dataset_readers.basic_classification_reader'['basic_classification_reader'] at line 112: Cannot find /content/drive/MyDrive/НИР/valid.csv file
2022-03-17 17:43:54.282 WARNING in 'deeppavlov.dataset_readers.basic_classification_reader'['basic_classification_reader'] at line 112: Cannot find /content/drive/MyDrive/НИР/test.csv file
2022-03-17 17:43:54.285 INFO in 'deeppavlov.dataset_iterators.basic_classification_iterator'['basic_classification_iterator'] at line 74: Splitting field <<train>> to new fields <<['train', 'valid', 'test']>>


In [ ]:
x_train, y_train = train_iterator.get_instances(data_type='train')

x_valid, y_valid = train_iterator.get_instances(data_type='valid')

x_test, y_test = train_iterator.get_instances(data_type='test')

In [ ]:
classifier_DB.preparation(x_train, y_train, x_valid, y_valid)
classifier_DB_ef.preparation(x_train, y_train, x_valid, y_valid)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [ ]:
all_valid_loss = []
all_train_loss = []
all_valid_acc = []
all_train_acc = []

valid_acc_ep, valid_loss_ep = classifier_DB_ef.eval()
predictions_wft = [str(classifier_DB_ef.predict(t)) for t in x_test]
all_valid_loss, all_valid_acc, all_train_loss, all_train_acc = classifier_DB.train('DistilRuBERT.pth')

Epoch 1/6
Train loss 0.07742643844124704 accuracy 0.976857142857143
Val loss 0.038489413494244215 accuracy 0.987
----------
Epoch 2/6
Train loss 0.01618617325677663 accuracy 0.9955714285714287
Val loss 0.05438544122024905 accuracy 0.9895
----------
Epoch 3/6
Train loss 0.004411280122357017 accuracy 0.9985714285714287
Val loss 0.05931754733758862 accuracy 0.9875
----------
Epoch 4/6
Train loss 0.0018342724131416533 accuracy 0.9995714285714287
Val loss 0.06107847631261393 accuracy 0.989
----------
Epoch 5/6
Train loss 0.0008705710174435973 accuracy 0.9997142857142858
Val loss 0.06201496029303234 accuracy 0.99
----------
Epoch 6/6
Train loss 0.0004906300813523209 accuracy 0.9998571428571429
Val loss 0.06182068876756239 accuracy 0.99
----------


In [ ]:
predictions = [str(classifier_DB.predict(t)) for t in x_test]

In [ ]:
precision, recall, f1score = precision_recall_fscore_support(y_test, predictions_wft)[:3]

print(f'precision: {precision}, recall: {recall}, f1score: {f1score}')

precision: [0.    0.477], recall: [0. 1.], f1score: [0.         0.64590386]


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [ ]:
precision, recall, f1score = precision_recall_fscore_support(y_test, predictions)[:3]

print(f'precision: {precision}, recall: {recall}, f1score: {f1score}')

precision: [0.98854962 0.9894958 ], recall: [0.99043977 0.98742138], f1score: [0.98949379 0.9884575 ]


In [ ]:
all_train_acc_pre = []
all_valid_acc_pre = []
valid_acc_pre = []

for i in range(6):
  all_train_acc_pre.append(all_train_acc[i].cpu())
  all_valid_acc_pre.append(all_valid_acc[i].cpu())

valid_loss = np.hstack((valid_loss_ep, all_valid_loss))
all_valid_acc_pre = np.array(all_valid_acc_pre)
valid_acc_ep = np.array(valid_acc_ep.cpu())
valid_acc = np.hstack((valid_acc_ep, all_valid_acc_pre))

In [ ]:
!pip install plotly

In [ ]:
import matplotlib.pyplot as plt 
import plotly.graph_objects as go

print('Originaly loss of the model:', valid_loss_ep)

fig = go.Figure(
    data=[go.Line(y = all_train_loss, x=[1,2,3,4,5,6])]
)
fig.update_layout(title='Train Loss DistilRuBERT',
                   xaxis_title='Epoch',
                   yaxis_title='Loss')

fig_2 = go.Figure(
    data=[go.Line(y = all_valid_loss, x=[1,2,3,4,5,6])]
)
fig_2.update_layout(title='Valid Loss DistilRuBERT',
                   xaxis_title='Epoch',
                   yaxis_title='Loss')

fig.show(renderer="colab")
fig_2.show(renderer="colab")


Originaly loss of the model: 0.7119246969223022


/usr/local/lib/python3.7/dist-packages/plotly/graph_objs/_deprecations.py:385: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.




In [ ]:
print('Originaly loss of the model:', valid_acc_ep)

fig = go.Figure(
    data=[go.Line(y = all_train_acc_pre, x=[1,2,3,4,5,6])]
)

fig.update_layout(title='Train Accuracy DistilRuBERT',
                   xaxis_title='Epoch',
                   yaxis_title='Accuracy')

fig_2 = go.Figure(
    data=[go.Line(y = all_valid_acc_pre, x=[1,2,3,4,5,6])]
)
fig_2.update_layout(title='Valid Accuracy DistilRuBERT',
                   xaxis_title='Epoch',
                   yaxis_title='Accuracy')

fig.show(renderer="colab")
fig_2.show(renderer="colab")


Originaly loss of the model: 0.486


/usr/local/lib/python3.7/dist-packages/plotly/graph_objs/_deprecations.py:385: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.


